<h1 style="text-align:center;font-size:40px;">Tools 1 Final Project</h1>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<p style="text-align:center;">Anthony Molieri<br>
Leo Walker</p>
<br>
<br>
<br>
<br>

# Dataset and Motivation

The data used for these analyses were scraped from Indeed and Glassdoor. The data from Indeed contains job descriptions for open Data Scientist positions. The Glassdoor data includes employee generated ratings on benefits and perks for many of the companies present in the Indeed dataset. The datasets were then combined in order to give an idea of how employees rank the benefits at these companies.

*The Indeed data was scraped over ... pulling job descriptions for Data Scientists in 25 cities throughout the US. **Leo** Add Correct Details, here about the Indded data and scraper*

The Glassdoor scraper was designed to pull company names from the the job description dataset. It then looped over the list of employers, searched, and scraped ratings from the benefits page for companies that were found. This data was then added to it's own table in the same database. The scraper was designed to be run multiple times, only searching for benefits ratings that were not already present in the database. 

# Task Definition and Research Question
 
The purpose of this project was to help give a better idea of how employees at companies with open positions rate their benefits. This can help give insight into how desirable a company is as an employer.

# Literature Review

*Add -- I'm not really sure what to add here. Maybe something about turnover in data science? Companies not having great benefits?*

# Data Cleaning

In [9]:
# imports
import numpy as np
import pandas as pd
import sqlite3
import re

In [10]:
# Initialize both tables
cnct = sqlite3.connect('Data/jobs')

indeed_df = pd.read_sql_query('SELECT * FROM indeed_jobs;', cnct)

glassdoor_df = pd.read_sql_query('SELECT companies, type, rating, count_of_ratings FROM ratings;', cnct)

## Cleaning Indeed Data

In [11]:
# # Pull missing salary data from description
# salaryFix = indeed_df[['job_desc', 'est_salary']].copy()

# salaryFix['check1'] = salaryFix['job_desc'].str.contains("\$")
# salaryFix['check2'] = salaryFix['job_desc'].str.contains("salary")

# I'm not great at regex but let's try this
# Only look for salary data when it's missint in 'est_salary' 
indeed_df['salary_extract'] = np.where(indeed_df['est_salary'] == "No Estimated Salary",indeed_df['job_desc'].str.extract("((\$)(([0-9])([0-9])([0-9])(,)([0-9])([0-9])([0-9])|([0-9])([0-9])(,)([0-9])([0-9])([0-9]))( to |-| - )(\$|)(([0-9])([0-9])([0-9])(,)([0-9])([0-9])([0-9])|([0-9])([0-9])(,)([0-9])([0-9])([0-9]))|(\$)([0-9][0-9][0-9],[0-9][0-9][0-9])|(\$)([0-9][0-9],[0-9][0-9][0-9]))")[0], np.NaN)

In [12]:
def set_job_category(job_title):
    regex = re.compile('[^a-zA-Z]')
    clean_title = regex.sub('', job_title)
    clean_title = clean_title.lower()
    if all(x in clean_title for x in ["data", "scien"]):
        return "Data Scientist"
    elif all(x in clean_title for x in ["business", "analy"]):
        return "Business Intelligence Analyst"
    elif all(x in clean_title for x in ["bi", "analy"]):
        return "Business Intelligence Analyst"
    elif all(x in clean_title for x in ["data", "analy"]):
        return "Data Analyst"
    elif all(x in clean_title for x in ["machine", "learning"]):
        return "Machine Learning Engineer"
    elif all(x in clean_title for x in ["data", "engineer"]):
        return "Data Engineer"
    elif all(x in clean_title for x in ["software", "engin"]):
        return "Software Engineer"
    elif all(x in clean_title for x in ["architect"]):
        return "Data Architect"
    else:
        return "Other"
indeed_df['job_category'] = indeed_df.loc[:, 'job_title'].apply(set_job_category)

In [20]:
indeed_df.to_csv('indeed.csv')

## Cleaning Glassdoor Data

# Visualizations